First, load the data,in this notebook we are using the Qitta data from the paper "Machine learning–based feature selection to search stable microbial biomarkers: application to inflammatory bowel disease". The data has been preprocessed in R (refer to the r script in iCloud/UTK/GRA-UTK/SelectMicro/MLonMicrobiome).

There are 3 datasets, the orginal paper combined the 3 datasets together, but not sure how they are combined since they have different columns. The imbalance is an issue if analyzing each dataset independently.

In [1]:
import sys
sys.path.append('./Code')
import loadData 
import RunML
import RunML_continue
import FS
import metric

import pandas as pd
import numpy as np
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm
import pickle

In [2]:
qitta_combine = pd.read_csv('./Data/Qitta_3datasetscombined_full.csv',index_col=0)

In [3]:
print(qitta_combine.shape)

(0, 15178)


In [4]:
print(qitta_combine)

Empty DataFrame
Columns: [Diagnosis, X4479946, X4307314, X4376890, X337735, X347991, X181348, X4454356, X261920, X4474844, X189592, X301062, X4455330, X301064, X3957916, X2506486, X300250, X4343580, X166156, X4465907, X4440096, X323271, X277265, X4371046, X4430921, X4454531, X4393783, X353886, X177792, X194101, X557576, X2700687, X297708, X4335436, X2996838, X576649, X336463, X336468, X295854, X3220253, X295856, X1974536, X298392, X4431292, X367150, X1135042, X176062, X3439275, X176067, X2851803, X269546, X1109552, X261210, X554372, X4352768, X4414820, X4414821, X4414822, X192795, X274018, X309054, X646961, X307256, X271880, X1111328, X345717, X1028632, X432284, X182061, X293285, X182065, X190485, X190332, X226125, X293289, X919158, X158711, X187248, X199301, X193829, X2865835, X3325760, X199307, X2689396, X558281, X694337, X182886, X321107, X269815, X573916, X190155, X4477450, X175267, X4480529, X564581, X660099, X185483, X1928156, X337909, X4442508, ...]
Index: []

[0 rows x 15178 co

Extract OTU table and response variable

In [5]:
data = qitta_combine.drop(qitta_combine.columns[0], axis=1)
data = data.values

In [6]:
print(data)
print(np.shape(data))

[]
(0, 15177)


In [7]:
data

array([], shape=(0, 15177), dtype=object)

In [8]:
y = qitta_combine.iloc[:, 0].values 

In [9]:
set(y)

set()

In [10]:
#change y into 2 levels
y = np.where(np.isin(y, ['CD', 'UC']), 'IBD', y)

In [11]:
y

array([], dtype=object)

In [12]:
# y is an array and y_lable is an series(pd.DataFrame)
y_lable = pd.DataFrame(y)
y_lable.value_counts()

Series([], Name: count, dtype: int64)

preprocess the OTU table

In [13]:
data=RunML.normalizingMatrixToRanks(data,cutOff=0.01)
print(np.shape(data))

(0, 15177)


Perform sigle lable feature selection, this is done by calculating Kruskal-Wallis H-test on each OTU for each environmental factor, each OTU/ASV is given a weighted h score.

In [14]:
weights=FS.singleLabelFeatureWeighting(data,y)

ERROR! Getting Non Binary Label Class!! 0 set()


TypeError: cannot unpack non-iterable NoneType object

In [ ]:
len(weights)

In [ ]:
scores=(sorted(weights,reverse=True))
eps=FS.elbowPoint(scores)
FS.plotWeightedIndex(weights,xKnee=eps)

This plot shows that the gain in information decreases as we include more OTUs along the ranks. 

In [ ]:
topFeature_selected=eps
print(eps)

X_FS,selectedOTU_index=FS.feature_select_singleLabel(data,y,topFeature=topFeature_selected)
print (np.shape(X_FS))


In [ ]:
print(len(weights))

Prepare 4 datasets: full dataset, our selected dataset, Lasso selected  dataset(based on the target variable), randomly selected data (selected the same numer of variables as in our method, repeat the  process iter=20 times to  find the mean accuracy and AUC)---> use random forest and SVM as classifier

In [ ]:
iter =10
cls = ["RF","SVM"]

In [ ]:
np.shape(y)

In [ ]:
targetLabel=y
X_lasso = RunML.LassoFeatureSelection(data,targetLabel)# this is depend on the target variable(do not update here)
data_subset = {"AllFeatures":data, 
               "SelectMicro": X_FS,
               "Lasso":X_lasso,
               "Random":data
              }


started modeling, the  function will print out the accuracy and AUC for each dataset using each classifier. and also will return the y_actual, y_predict, y_predprob

In [ ]:
dict_cm = RunML_continue.runClassifier_FScompare(data_subsets= data_subset,y= targetLabel,N=iter,classifiers=cls)

In [ ]:
# Save dictionary to a pickle file
with open('results/dict_cm_qitta.pkl', 'wb') as pickle_file:
    pickle.dump(dict_cm, pickle_file)

In [ ]:
# Load dictionary from the pickle file
with open('results/dict_cm_qitta.pkl', 'rb') as pickle_file:
    dict_cm = pickle.load(pickle_file)

In [ ]:
# do not use plotmacro_confusion_matrices (not for binary classification)
for dataset_name, classifiers in dict_cm.items():
        for classifier_name, labels in classifiers.items():
            actual_labels = labels[0]
            predicted_labels = labels[1]
            metric.plotmacro_confusion_matrices(actual_labels, predicted_labels,f"{dataset_name} - {classifier_name}")

## compare the first 15 index

In [ ]:
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

entries=15
selectedOTU_index_15=selectedOTU_index[:entries]
X_FS_15=data[:,selectedOTU_index_15]

df=pd.DataFrame(data=X_FS_15)
selectedASVs=[ASVs[i] for i in selectedOTU_index_15]


In [ ]:
print(set(targetLabel))
RunML.plotPresenseRatio(X_FS_15,targetLabel,selectedASVs,posLabel="IBD",posText="IBD",negText="nonIBD",entries=entries)